### Python的内建模块itertools提供了非常有用的用于操作迭代对象的函数。

In [2]:
import itertools
natu = itertools.count(1)
ns = itertools.takewhile(lambda x: x <=10, natu)
list(ns)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [3]:
# itertools提供的几个迭代器操作函数更加有用：
# chain()可以把一组迭代对象串联起来，形成一个更大的迭代器：
for x in itertools.chain('abc', 'def', 'hig'):
    print(x)

a
b
c
d
e
f
h
i
g


In [4]:
# groupby()把迭代器中相邻的重复元素挑出来放在一起：
# 注意是相邻的重复元素分在一起，不相邻的重复元素不在一起
for key, group in itertools.groupby('AAABBBCCAAA'):
    print(key, list(group))


A ['A', 'A', 'A']
B ['B', 'B', 'B']
C ['C', 'C']
A ['A', 'A', 'A']


In [5]:
for k, g in itertools.groupby('AAABbBCCcCAaa', lambda c: c.upper()):
    print(k, list(g))


A ['A', 'A', 'A']
B ['B', 'b', 'B']
C ['C', 'C', 'c', 'C']
A ['A', 'a', 'a']


#### 并不是只有open()函数返回的fp对象才能使用with语句。实际上，任何对象，只要正确实现了上下文管理，就可以用于with语句。
#### 实现上下文管理是通过__enter__和__exit__这两个方法实现的。例如，下面的class实现了这两个方法：

In [6]:
class Query(object):
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        print('begin')
        return self
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('End')
    def query(self):
        print('Query info about %s' % self.name)
        
with Query('Tom') as q:
    q.query()

begin
Query info about Tom
End


#### 编写__enter__和__exit__仍然很繁琐，因此Python的标准库contextlib提供了更简单的写法，上面的代码可以改写如下：

In [7]:
from contextlib import contextmanager
class Query(object):

    def __init__(self, name):
        self.name = name

    def query(self):
        print('Query info about %s...' % self.name)
@contextmanager
def create_query(name):
    print('Begin')
    q = Query(name)
    yield q
    print('End')

# @contextmanager这个decorator接受一个generator，用yield语句把with ... as var把变量输出出去，然后，with语句就可以正常地工作了：

with create_query('Tom') as q:
    q.query()


Begin
Query info about Tom...
End


In [8]:
@contextmanager
def tag(name):
    print('<%s>' % name)
    yield
    print('<%s>' % name)

with tag('h1'):
    print('hello')
    print('world')
    

<h1>
hello
world
<h1>


#### 代码的执行顺序是：
    - with语句首先执行yield之前的语句，因此打印出<h1>；
    - yield调用会执行with语句内部的所有语句，因此打印出hello和world；
    - 最后执行yield之后的语句，打印出</h1>。

In [11]:
import psutil
print(psutil.cpu_count()) ## CPU逻辑数量
print(psutil.cpu_count(logical=False))  #CPU物理核心

8
4


In [12]:
psutil.virtual_memory()

svmem(total=8589934592, available=2294960128, percent=73.3, used=4853755904, free=171053056, active=2501922816, inactive=1884651520, wired=2351833088)

In [13]:
psutil.swap_memory()

sswap(total=1073741824, used=51904512, free=1021837312, percent=4.8, sin=11708411904, sout=88580096)

#### 网络通信实际上是两台计算机上的两个进程之间在通信


In [15]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# 创建Socket时，AF_INET指定使用IPv4协议，如果要用更先进的IPv6，就指定为AF_INET6。SOCK_STREAM指定使用面向流的TCP协议，
s.connect(('www.sina.com.cn', 80))
# 发送数据
s.send(b'GET / HTTPS/1.1\r\nHost: www.sina.com.cn\r\nConnection: close\r\n\r\n')
# 接收数据
buffer = []
while True:
    # 每次最多接收1k字节:
    d = s.recv(1024)
    if d:
        buffer.append(d)
    else:
        break
data = b''.join(buffer)
# 关闭连接
s.close()

header, html = data.split(b'\r\n\r\n', 1)
print(header.decode('utf-8'))
with open('sina.html', 'wb') as f:
    f.write(html)
    

HTTP/1.1 400 Bad Request
Server: nginx
Date: Tue, 07 May 2019 14:42:25 GMT
Content-Type: text/html
Content-Length: 166
Connection: close
X-Via-CDN: f=edge,s=cmcc.hangzhou.ha2ts4.75.nb.sinaedge.com,c=39.186.188.95;


### Socket又称"套接字"，应用程序通常通过"套接字"向网络发出请求或者应答网络请求，使主机间或者一台计算机上的进程间可以通讯。

In [ ]:
# 一封电子邮件的旅程就是
# 发件人 -> MUA -> MTA -> MTA -> 若干个MTA -> MDA <- MUA <- 收件人

#### 有了上述基本概念，要编写程序来发送和接收邮件，本质上就是：

- 编写MUA把邮件发到MTA
- 编写MUA从MDA上收邮件

- 发邮件时，MUA和MTA使用的协议就是SMTP：Simple Mail Transfer Protocol，后面的MTA到另一个MTA也是用SMTP协议。


- 收邮件时，MUA和MDA使用的协议有两种：
 - POP：Post Office Protocol，目前版本是3，俗称POP3；
 - IMAP：Internet Message Access Protocol，目前版本是4，优点是不但能取邮件，还可以直接操作MDA上存储的邮件，比如从收件箱移到垃圾箱。

